In [166]:
import requests
import json
import boto3
from botocore.exceptions import NoCredentialsError
from datetime import datetime
import os
import sys
import re
import yaml

In [167]:
# MinIO Storage Settings
MINIO_ENDPOINT = os.getenv("AWS_S3_ENDPOINT")
MINIO_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
MINIO_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
PLAYBOOKS_BUCKET = "playbook"
LOG_FILE = "service_error_down.txt" # This variable will be used throughout the script

# LLM Info
INFERENCE_ENDPOINT = "https://granite-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com"
MODEL_API_URL = f"{INFERENCE_ENDPOINT}/v1/completions"
MODEL_NAME = "granite"

In [168]:
# Helper Functions

def extract_and_save_keywords(report_filename="incident_report.txt", output_filename="extracted_keywords.json"):
    """Reads the full report, extracts keywords, and saves them to a new file."""
    print(f"\n--> Step 1: Extracting and saving keywords...")
    details = {}

    with open(report_filename, 'r', encoding='utf-8') as f:
        report_text = f.read()
    # Define regex patterns for host, service, and optionally a port
    host_pattern = re.search(r"AFFECTED HOST:\s*(.*)", report_text, re.IGNORECASE)
    service_pattern = re.search(r"AFFECTED SERVICE:\s*(.*)", report_text, re.IGNORECASE)
    port_pattern = re.search(r"port (\d+)", report_text, re.IGNORECASE)

    if host_pattern:
        details["host"] = host_pattern.group(1).strip().strip('*').strip()
    if service_pattern:
        raw_service = service_pattern.group(1).strip().strip('*').strip()
        details["service"] = raw_service.replace('.service', '') if raw_service.endswith('.service') else raw_service
        
    # Set a default port and override if found in the text
    details["port"] = 80
    if port_pattern:
        details["port"] = int(port_pattern.group(1))
        
    if "host" in details and "service" in details:
        print(f"    ✅ Extracted Host: '{details['host']}', Service: '{details['service']}', Port: {details['port']}")
        with open(output_filename, 'w') as out_f:
            json.dump(details, out_f, indent=4)
        print(f"    ✅ Successfully saved keywords to '{output_filename}'.")
        return details
        
    print("    ❌ Failed to extract keywords.")
    return None

def upload_to_minio(s3_client, bucket, object_name, content):
    """Uploads content to a specified MinIO bucket."""
    print(f"--> Uploading '{object_name}' to bucket '{bucket}'...")
    try:
        s3_client.put_object(Body=content.encode('utf-8'), Bucket=bucket, Key=object_name)
        print(f"    ✅ Successfully uploaded '{object_name}'.")
        return True
    except Exception as e:
        print(f"    ❌ Error uploading to MinIO: {e}")
        return False


In [157]:
# Extract keywords into dictionary from incident report
keywords = extract_and_save_keywords()

print(keywords)


--> Step 1: Extracting and saving keywords...
    ✅ Extracted Host: 'aiops', Service: 'httpd', Port: 80
    ✅ Successfully saved keywords to 'extracted_keywords.json'.
{'host': 'aiops', 'service': 'httpd', 'port': 80}


In [178]:
# Prompt LLM

with open("incident_report.txt", 'r', encoding='utf-8') as f:
    report_content = f.read()

prompt = f"""
You are an Ansible playbook writing assistant.

Generate a **complete Ansible playbook** in valid YAML format.
The playbook must include:
- A single play with the following header:
  - name: Restore {keywords.get('service', 'service').upper()}
  - hosts: {keywords.get('host')}
  - become: true
  - vars:
      max_retries: 3
      retry_delay: 10
      service: {keywords.get('service')}
      port: {keywords.get('port')}
- A tasks section with appropriate remediation tasks based on the incident report below.

Guidelines:
- Output only valid YAML, no markdown or extra explanations.
- Do not use markdown formatting like ```yaml.
- Use appropriate Ansible modules (e.g., ansible.builtin.service, command, shell, assert).
- All tasks must be properly indented under the `tasks:` key.

**Incident Report:**
{report_content}
"""

headers = {"Content-Type": "application/json"}
payload = {"model": MODEL_NAME, "prompt": prompt, "max_tokens": 1024}

response = requests.post(MODEL_API_URL, headers=headers, json=payload, verify=False)
response.raise_for_status()
result = response.json()

if 'choices' in result and result['choices']:
    playbook_yaml = result['choices'][0].get('text', '').strip()  
    if "```" in playbook_yaml:
        playbook_yaml = re.search(r"```(?:\w*\n)?(.*)```", playbook_yaml, re.DOTALL).group(1)
    print("    ✅ AI-generated tasks received.")
    print(playbook_yaml)


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'granite-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


    ✅ AI-generated tasks received.
- name: Restore HTTPD
  hosts: aiops
  become: true
  vars:
    max_retries: 3
    retry_delay: 10
    service: httpd
    port: 80
  tasks:
    - name: Start the HTTP server service
      ansible.builtin.service:
        name: "{{ service }}"
        state: started
        enabled: yes

    - name: Check the status of the HTTP server service
      ansible.builtin.command: systemctl status "{{ service }}"

    - name: Verify that the HTTP server is listening on Port 80
      ansible.builtin.command: ss -tln | grep "{{ port }}"

    - name: Assert that the HTTP server service is running
      ansible.builtin.assert:
        that:
          - systemctl status "{{ service }}" | grep -q "active (running)"
        fail_msg: "HTTP server service is not running"
        success_msg: "HTTP server service is running"

    - name: Assert that the HTTP server is listening on Port 80
      ansible.builtin.command: ss -tln | grep "{{ port }}"
      register: port_c

In [177]:
# Write playbook response to file 
output_file = "playbook.yml"
with open(output_file, "w") as f:
    f.write(playbook_yaml)


# Upload playbook yaml to s3
s3_client = boto3.client(
    's3', 
    endpoint_url=MINIO_ENDPOINT, 
    aws_access_key_id=MINIO_ACCESS_KEY, 
    aws_secret_access_key=MINIO_SECRET_KEY
    )

with open(output_file, 'r', encoding='utf-8') as f:
    output_file = f.read()

upload_to_minio(s3_client, PLAYBOOKS_BUCKET, f"remediation_playbook_{datetime.now().strftime('%Y%m%d_%H%M%S')}.yaml", output_file)

--> Uploading 'remediation_playbook_20250617_143733.yaml' to bucket 'playbook'...
    ✅ Successfully uploaded 'remediation_playbook_20250617_143733.yaml'.


True